In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        break

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam

import pandas as pd
import numpy as np
import pathlib
import os
import copy
import random
import time
import matplotlib.pyplot as plt
import matplotlib.image as img
from random import randint
from PIL import Image
from IPython.display import HTML
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# dataset path
data_dir = pathlib.Path('../input/leaf-images/300_dataset')
data_dir

In [ ]:
d = {
    'path': ['../input/leaf-images/300_dataset/'] ,
    'pathlib': [data_dir] ,
    'apta': [list(data_dir.glob('Apta/*'))[:1][0]] ,
    'indian_rubber_tree': [list(data_dir.glob('Indian Rubber Tree/*'))[:1][0]] ,
    'karanj': [list(data_dir.glob('Karanj/*'))[:1][0]] ,
    'kashid': [list(data_dir.glob('Kashid/*'))[:1][0]] ,
    'nilgiri': [list(data_dir.glob('Nilgiri/*'))[:1][0]],
    'pimpal': [list(data_dir.glob('Pimpal/*'))[:1][0]],
    'sita_ashok': [list(data_dir.glob('Sita Ashok/*'))[:1][0]],
    'sonmohar': [list(data_dir.glob('Sonmohar/*'))[:1][0]],
    'vad': [list(data_dir.glob('Vad/*'))[:1][0]],
    'vilayati_chinch': [list(data_dir.glob('Vilayati Chinch/*'))[:1][0]]
}

df = pd.DataFrame(data = d)
df.to_csv('info.csv')
df

In [ ]:
#assigning classes and labels
imgClass = ['Apta', 'Indian Rubber Tree', 'Karanj', 'Kashid', 'Nilgiri',  'Pimpal', 'Sita Ashok', 'Sonmohar', 'Vad', 'Vilayati Chinch']
imgCat = ['apta','indian_rubber_tree','karanj','kashid','nilgiri', 'pimpal', 'sita_ashok', 'sonmohar', 'vad', 'vilayati_chinch']
imgLabel = {
    'Apta' : 0,
    'Indian Rubber Tree' : 1,
    'Karanj' : 2,
    'Kashid' : 3,
    'Nilgiri': 4,
    'Pimpal': 5,
    'Sita Ashok': 6,
    'Sonmohar': 7,
    'Vad': 8,
    'Vilayati Chinch': 9
}

In [ ]:
def makeDirectDownload(title = "Download File", filename = "ai.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

In [ ]:
#view images
fig, ax = plt.subplots(ncols = 10, figsize = (27,5))
fig.suptitle('Leaf Category')
i = 0

for leafs in imgCat:
    ax[i].set_title(leafs)
    ax[i].imshow(img.imread(df[leafs].values[0]))
    i+=1
    
plt.show()

In [ ]:
# configuring image classes
path = df.path.values[0]
lenClass = len(imgClass)
print(lenClass)
files = [[os.path.join(path, Class, x) for x in os.listdir(os.path.join(path, Class))] for Class in imgClass]

In [ ]:
# some images seem to be corrupted so we are only going to use the non corrupt ones


imgPaths = []
for i in range(10):
    for j in range(len(files[i])):
        paths = files[i]
        filepath = paths[j]
        if filepath.endswith(".jpg") or filepath.endswith(".jpeg") or filepath.endswith(".png"):
            # check if the image is corrupt
            try:
                with Image.open(filepath) as img:
                    img.verify()
                    imgPaths.append(paths[j])
            except:
                # if the image is corrupt, skip it
                pass

In [ ]:
random.shuffle(imgPaths)

In [ ]:
# image augmentation
transformations = transforms.Compose(
    [
        transforms.Resize((256,256)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize((0.5),(0.5))
    ]
)

In [ ]:
# configuring image dataset
class imgDataset(Dataset):
    def __init__(self, df_labels, base_dir, transform=None):
        super().__init__()
    
        self.base_dir = base_dir
        self.df_labels = df_labels
        self.transform = transform
        
    def __len__(self):
        return len(self.base_dir)
    
    def __getitem__(self, i):
        
        imagePath = self.base_dir[i]
        Img = Image.open(imagePath)
        labelName = imagePath.split('/')[-2]
#         print(labelName)
        label = self.df_labels[labelName]
        
        if self.transform is not None:
            Img = self.transform(Img)
        return (Img, label)

In [ ]:
# split dataset
train_paths = imgPaths[:2100]
test_paths = imgPaths[2100:2700]
valid_paths = imgPaths[2700:]

In [ ]:
train_data = imgDataset(imgLabel, train_paths, transformations)
test_data = imgDataset(imgLabel, test_paths, transformations)
valid_data = imgDataset(imgLabel, valid_paths, transformations)

In [ ]:
train_ds = DataLoader(train_data, batch_size = 64, shuffle = True)
val_ds = DataLoader(valid_data, batch_size = 64, shuffle = True)
test_ds = DataLoader(test_data, batch_size = 64, shuffle = True)

In [ ]:
class LeNet(nn.Module):
    def __init__(self, output_dim):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=16,kernel_size=5)

        self.fc_1 = nn.Linear(16 * 61 * 61, 120)
        self.fc_2 = nn.Linear(120, 84)
        self.fc_3 = nn.Linear(84, output_dim)

    def forward(self, x):
    
        x = self.conv1(x) # input 3, 256, 256
        x = F.max_pool2d(x, kernel_size=2) # output 6, 252, 252

        x = F.relu(x) # 6, 126, 126
        x = self.conv2(x)

        x = F.max_pool2d(x, kernel_size=2) # 16, 122, 122

        x = F.relu(x) # 16, 61, 61
        x = x.view(x.shape[0], -1)
        h = x
        x = self.fc_1(x)
        x = F.relu(x)
        x = self.fc_2(x)
        x = F.relu(x)
        x = self.fc_3(x)
        return x, h

In [ ]:
# number of leaf classes
OUTPUT_DIM = 10
model = LeNet(OUTPUT_DIM)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Trainable Parameters = {count_parameters(model):,}')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
model = model.to(device)
print(model)

In [ ]:
optimizer = Adam(model.parameters(), lr = 0.0002)
loss_fun = F.cross_entropy

In [ ]:
_train_loss=[]
_train_acc=[]
_predict=[]
_correct_num=[]

_eval_loss=[]
_eval_acc=[]
_evalPredict=[]
_evalCorrect_num=[]
_evalLabel=[]

_image = []

In [ ]:
!nvidia-smi

In [ ]:
def train(model, epoch, train_ds):
#     model.to(torch.device("cuda"))
    model.train()
    total_num = len(train_ds.dataset)
    train_loss = 0
    correct_num = 0

    for image, label in train_ds:
        image = image.to(device)
        label = label.to(device)

        # Convert the tag from int32 type to long type, otherwise the calculation loss will report an error
        label = label.to(torch.long)

        output,_ = model(image)
        loss = loss_fun(output, label)
        train_loss += loss.item() * label.size(0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predict = torch.argmax(output, dim=-1)
        correct_num += label.eq(predict).sum()

    train_loss = train_loss / total_num
    train_acc = correct_num / total_num
    
    _train_loss.append(train_loss)
    _train_acc.append(train_acc)
    _predict.append(predict)
    _correct_num.append(correct_num)
    
    print('Epoch: {} >>> TrainLoss: {:.6f} - TrainAcc: {:.6f} - '.format(epoch, train_loss, train_acc), end='')
    
    
def evaluate(model, eval_ds, mode='val'):
    model.eval()

    total_num = len(eval_ds.dataset)
    eval_loss = 0
    correct_num = 0

    for image, label in eval_ds:

        _image.append(image)

        image = image.to(device)
        label = label.to(device)
        _evalLabel.append(label)
        
        label = label.to(torch.long)
        
        output,_ = model(image)

        loss = loss_fun(output, label)
        eval_loss += loss.item() * label.size(0)

        predict = torch.argmax(output, dim=-1)
        correct_num += label.eq(predict).sum()
    
    eval_loss = eval_loss / total_num
    eval_acc = correct_num / total_num
    
    _eval_loss.append(eval_loss)
    _eval_acc.append(eval_acc)
    _evalPredict.append(predict)
    _evalCorrect_num.append(correct_num)
    
    print('{}_Loss: {:.6f} - {}_Acc: {:.6f}'.format(mode, eval_loss, mode, eval_acc))

In [ ]:
ep = 100
for epoch in range(ep):
    train(model, epoch, train_ds)
    evaluate(model, val_ds)

torch.save(model.state_dict(),'./model{}.pt'.format(ep))